In [25]:
from PIL import Image
#from itertools import izip
import hashlib
import base64

from Crypto import Random
from Crypto.Cipher import AES

#The get_pixel_pairs function takes in our pixel list and returns the pairs back; 
#as each message character was split over two pixels, this makes extraction easier. 
#The other helper function get_LSB will take in an R, G, B, or A value and 
#use a bit mask to get the LSB value and return it in a string format

def get_pixel_pairs(iterable):
    a = iter(iterable)
    return zip(a, a)


def get_LSB(value):
    if value & 1 == 0:
        return '0'
    else:
        return '1'
#carrier is the input image
def extract_message(carrier):
    c_image = Image.open(carrier)
    pixel_list = list(c_image.getdata())
    message = ""

    for pix1, pix2 in get_pixel_pairs(pixel_list):
        #The next part of code that we will create is a string variable that will hold our binary string. 
        #Python knows that it'll be the binary representation of a string by the 0b prefix.
        #We then iterate over the RGBA values in each pixel (pix1 and pix2) and pass that value 
        #to our helper function, get_LSB, the value that's returned is appended onto our binary string:
        message_byte = "0b"
        for p in pix1:
            message_byte += get_LSB(p)

        for p in pix2:
            message_byte += get_LSB(p)

        if message_byte == "0b00000000":
            break

        message += chr(int(message_byte,2))
    return message

def unpad(s):
    return s[0:-s[-1]]
#This is same as that of encryption
class AESCipher:

    def __init__( self, key ):
        self.bs = 32
        self.key = hashlib.sha256(key.encode()).digest()

    def decrypt( self, enc ):
        enc = base64.b64decode(enc)
        iv = enc[:16]
        cipher = AES.new(self.key, AES.MODE_CBC, iv )
        return unpad(cipher.decrypt( enc[16:] ))


def main():
    img_path = input("Enter the path of the image:")
    decryp_key = input("Enter decryption key:")
    try:
        cipher = AESCipher(decryp_key)#creating an instance for AESCipher
        encryp_msg = extract_message(img_path)#Extracts the encrypted hidden message from the image in the form of string
        encry = bytes(encryp_msg[2:len(encryp_msg)-1],'utf-8')#converting string to bytes
        decrypted = cipher.decrypt(encry)#Decrypting the encrypted message using dcrypt() function
        decr_str = decrypted.decode("utf-8")#decoding it into string
        print(decr_str)
    except:
        print("Please check your entries again")
    
#Interpretation starts here    
main()

Enter the path of the image:school_cipher.png
Enter decryption key:lalap@189
Hello there! I'm currently on my internship period
